### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
#Num players based on unique SN
numPlayers = int(len(purchase_data["SN"].unique()))
totalPlayers_df = pd.DataFrame({"Total Players": [numPlayers]})
totalPlayers_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
#Number of items based on unique ID
numUniqueItems = len(purchase_data["Item ID"].unique())
#mean of all prices of purchased items
avgPrice = (purchase_data["Price"].mean())
numPurchases = len(purchase_data["Purchase ID"])
totRevenue = round(purchase_data["Price"].sum(),2)

#Inserting info into dataframe
purAnalysis_df = pd.DataFrame ({"Number of Unique Items": [numUniqueItems],
                               "Average Price": avgPrice,
                               "Number of Purchases": [numPurchases],
                               "Total Revenue": totRevenue})
#format to style
purAnalysis_df = purAnalysis_df.style.format({"Average Price": '${:.3}',"Total Revenue": '${:,}'})

purAnalysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#Extracting columns for usage from Purchase_data csv
gender = purchase_data[["SN","Gender","Age"]].drop_duplicates("SN")
genderCounts = gender["Gender"].value_counts()
perMale = genderCounts[0]/numPlayers
perFemale = genderCounts[1]/numPlayers
perOther = genderCounts[2]/numPlayers


#inserting calculations into dataframe
genderdf = pd.DataFrame([genderCounts]).T.rename(columns= {'Gender':'Total Counts'})
genderdf['Percentage of Players'] = [perMale, perFemale, perOther]
genderdf= genderdf.style.format({'Percentage of Players':'{:.2%}'})
genderdf

,Total Counts,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
#multiple dataframes for each calculation (using gender as a Key) and using count, mean, sum to calculate
purCount = pd.DataFrame(purchase_data.groupby('Gender', as_index = False).Price.count().rename(columns = {'Price':'Purchase Count'}))
averagePr = pd.DataFrame(purchase_data.groupby('Gender', as_index = False).Price.mean().rename(columns = {'Price':'Average Purchase Price'}))
totPr = pd.DataFrame(purchase_data.groupby('Gender',as_index = False).Price.sum().rename(columns = {'Price':'Total Purchase Value'}))

#merging all the dataframes into one
pA = pd.merge(purCount, averagePr, on = 'Gender')
pA = pd.merge(pA, totPr, on = 'Gender')

#formatting and final dataframe merge 
gendertest = pd.DataFrame([genderCounts]).T
avgTotal = pd.merge(gendertest, totPr, left_index = True, right_on = 'Gender')
avgTotal['avg'] = avgTotal.iloc[:,3]/avgTotal.iloc[:,1]
pA['Avg Total Purchase per Person'] = avgTotal.iloc[:,4]
pA= pA.set_index('Gender')
pA = pA.style.format({pA.columns[1]:'${:.2f}',pA.columns[2]:'${:,.2f}',pA.columns[3]:'${:.2f}'})
pA

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
#binning the age and creating labels
ageBin = [0, 9.9, 14.9,19.9,24.9,29.9,34.9,39.9,1000]
ageLabels = ['<10', '10-14','15-19','20-24','25-29','30-34','35-39','40+']

#pulling information from gender
agedf = gender[['SN','Age']]
agedf["Range"] = pd.cut(gender["Age"], ageBin, labels=ageLabels, include_lowest=True)

#SN column is not needed (used as placeholder to check info)
del agedf["SN"]

#grouping by Binned group
ageDemo = agedf.groupby(['Range']).count().rename(columns = {'Age':'Total Count'})

#creating percentage of Players and formatting
ageDemo['Percentage of Players'] = (ageDemo.iloc[:,]/ageDemo.sum())
ageDemographic = ageDemo.style.format({ageDemo.columns[1]:'{:.2%}'})

ageDemographic

,Total Count,Percentage of Players
Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
#exctracting data needed this time with Age and Price
agePur = purchase_data[["Age","Price"]]

#using previous binned info, creating a column for Range
agePur["Range"] = pd.cut(agePur["Age"], ageBin, labels=ageLabels, include_lowest=True)
ageCount = agePur.groupby('Range', as_index = False).Price.count().rename(columns = {'Price':'Purchase Count'})
ageAvgPur = agePur.groupby('Range', as_index = False).Price.mean().rename(columns = {'Price':'Average Purchase Price'})
ageTotPur = agePur.groupby('Range', as_index = False).Price.sum().rename(columns = {'Price':'Total Purchase Value'})
ageAvgTot = pd.merge(ageTotPur,ageDemo, on = 'Range')
#no need for percentage of players and total count
del ageAvgTot['Percentage of Players']
ageAvgTot['Avg Total Purchase per Person'] = ageAvgTot.iloc[:,1]/ageAvgTot.iloc[:,2]
del ageAvgTot['Total Count']


#merging all created dataframe assets into one and renaming column. Setting the index to the ranges
aPa = pd.merge(pd.merge(ageCount, ageAvgPur, on = "Range"),ageAvgTot, on = "Range").rename(columns = {'Range':'Age Ranges'})
aPa = aPa.set_index('Age Ranges')

#formatting the 2,3,4 column (alternatively could use iloc and save space)
agePurchaseAnalysis = aPa.style.format({aPa.columns[1]:'${:.2f}',aPa.columns[2]:'${:,.2f}',aPa.columns[3]:'${:.2f}'})
agePurchaseAnalysis

<ipython-input-7-bc30d1e6eed1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agePur["Range"] = pd.cut(agePur["Age"], ageBin, labels=ageLabels, include_lowest=True)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
#Pulling data needed
SNinfo = purchase_data[['SN','Price','Purchase ID']]
#calculations into data frames
SNcount = pd.DataFrame(SNinfo['SN'].value_counts()).rename(columns={'SN':'Purchase Count'})
SNavgpur = SNinfo.groupby('SN', as_index = False).Price.mean().rename(columns = {'Price':'Average Purchase Price'})
TopSpenders = pd.merge(SNcount, SNavgpur, left_index = True, right_on = 'SN')
TopSpenders['Total Purchase Value'] = TopSpenders.iloc[:,0]*TopSpenders.iloc[:,2]

#sorting by top spenders and setting index. formatting columns using apply map (multiple columns)
TopSpenders = TopSpenders.sort_values('Total Purchase Value', ascending = 0).set_index('SN')
TopSpenders.iloc[:,1:3] = TopSpenders.iloc[:,1:3].applymap('${:.2f}'.format)

#display preview
TopSpenders.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
#extracting relevant info
itemInfo = purchase_data[['Item ID','Item Name','Price']]

#performed calculations using item name as key (could use item ID, but wanted to use Item Name to remove confusion from index)
itemcount = pd.DataFrame(itemInfo['Item Name'].value_counts()).rename(columns={'Item Name':'Purchase Count'})
itemPrice = itemInfo.groupby('Item Name', as_index = False).Price.mean().rename(columns = {'Price':'Average Purchase Price'})
popItems = pd.merge(itemcount, itemPrice, left_index = True, right_on = 'Item Name')
popItems['Total Purchase Value']= popItems.iloc[:,0]*popItems.iloc[:,2]

#merging ID with those names
popItems = pd.merge(popItems, (itemInfo[['Item Name','Item ID']].drop_duplicates('Item Name')), on = 'Item Name', how = 'left').set_index(['Item ID','Item Name'])

#sorting the final into a seperate Dataframe to be viewed (need to save the popItems for later use)
mostItem = popItems.sort_values('Purchase Count', ascending = 0)
mostItem.iloc[:,1:3] = popItems.iloc[:,1:3].applymap('${:.2f}'.format)
mostItem.head(5)



,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
132,Persuasion,9,$3.22,$28.99
82,Nirvana,9,$4.90,$44.10
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
#using popItems saved, sorting based on total purchase value in descending order 
#formatted and displayed 
mostProfit = popItems.sort_values('Total Purchase Value', ascending = 0)
mostProfit.iloc[:,1:3] = mostProfit.iloc[:,1:3].applymap('${:.2f}'.format)
mostProfit.head(5)

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
